In [ ]:
import tqdm

file_path = "ol_dump_editions_2024-12-19.txt"  # Update with actual file path

# Estimate total rows for progress tracking
print("Estimating total rows...")
with open(file_path, "r", encoding="utf-8") as f:
    total_rows = sum(1 for _ in f)

print(f"Total estimated rows: {total_rows:,}")


In [28]:
import duckdb
import json
import pandas as pd

# Path to your large Open Library dump file
file_path = "ol_dump_2025-01-08.txt"  # Update with actual path
# select rows with type /type/edition

# Connect to DuckDB in memory
conn = duckdb.connect()

# Extract a 10-row sample
sample_query = f"""
SELECT * FROM read_csv_auto('{file_path}', delim='\t', header=False, columns={{
    'type': 'TEXT', 'book_id': 'TEXT', 'revision': 'INTEGER', 'timestamp': 'TEXT', 'json_data': 'TEXT'
}})
WHERE type = '/type/edition'
LIMIT 10
"""

# Fetch the sample data
sample_df = conn.execute(sample_query).fetchdf()


In [29]:
sample_df

,type,book_id,revision,timestamp,json_data
0,/type/edition,/books/OL10000299M,3,2011-04-27T14:29:38.147197,"{""publishers"": [""Stationery Office Books""], ""p..."
1,/type/edition,/books/OL10000402M,3,2011-04-27T11:28:52.494785,"{""publishers"": [""Stationery Office Books""], ""p..."
2,/type/edition,/books/OL10000661M,3,2011-04-25T22:07:29.549200,"{""publishers"": [""Stationery Office Books""], ""p..."
3,/type/edition,/books/OL10000852M,3,2011-04-27T04:54:31.855299,"{""publishers"": [""Stationery Office Books""], ""p..."
4,/type/edition,/books/OL10001003M,3,2011-06-08T04:28:03.025393,"{""publishers"": [""Stationery Office Books""], ""p..."
5,/type/edition,/books/OL10001092M,3,2011-04-28T09:07:33.035640,"{""publishers"": [""Stationery Office Books""], ""p..."
6,/type/edition,/books/OL10001125M,3,2011-04-26T15:08:38.906519,"{""publishers"": [""Stationery Office Books""], ""p..."
7,/type/edition,/books/OL10001268M,3,2011-04-26T18:02:19.907101,"{""publishers"": [""Stationery Office Books""], ""p..."
8,/type/edition,/books/OL10001455M,3,2022-07-17T23:09:07.154959,"{""publishers"": [""Stationery Office""], ""languag..."
9,/type/edition,/books/OL10001465M,3,2022-07-17T22:01:18.764450,"{""publishers"": [""Stationery Office""], ""languag..."


In [ ]:
sample_df["json_data"] = sample_df["json_data"].apply(lambda x: json.loads(x) if isinstance(x, str) else {})

# Normalize the JSON into a structured format
normalized_df = pd.json_normalize(sample_df["json_data"])

# Merge the normalized JSON with the original DataFrame (excluding raw JSON column)
final_df = pd.concat([sample_df.drop(columns=["json_data"]), normalized_df], axis=1)

# Display the structured DataFrame
final_df

In [1]:
import duckdb
import json
import pandas as pd
from tqdm import tqdm
import os

In [30]:
import duckdb
import pandas as pd
from tqdm import tqdm

file_path = "ol_dump_2025-01-08.txt"  # Update with actual path
output_file = "processed_open_library_data_dump.csv"

conn = duckdb.connect()

chunk_size = 100000
processed_rows = 0
first_write = True


In [10]:
print("Estimating total rows...")
try:
    with open(file_path, "r", encoding="utf-8") as f:
        total_rows = sum(1 for _ in f)
    print(f"Total estimated rows: {total_rows:,}")

Estimating total rows...
Total estimated rows: 53,073,982


In [33]:
import time  # Import the time module for sleep functionality

processed_rows = 0
sleep_time = 1  # Set the sleep time in seconds (adjust as needed)

while True:
    query = f"""
    SELECT
        book_id, revision, timestamp, json_data
    FROM read_csv_auto('{file_path}',
        delim='\t',
        header=False,
        max_line_size=10000000,  -- Increase the limit to just above the largest line size
        columns={{
            'type': 'TEXT',
            'book_id': 'TEXT',
            'revision': 'BIGINT',  -- Use BIGINT to match sniffed type
            'timestamp': 'TIMESTAMP',  -- Use TIMESTAMP to match sniffed type
            'json_data': 'TEXT'
        }}
    )
    WHERE type = '/type/edition'
    LIMIT {chunk_size} OFFSET {processed_rows}
    """

    try:
        chunk_df = conn.execute(query).fetchdf()
    except duckdb.BinderException as e:
        print(f"Query execution failed: {e}")
        break

    if chunk_df.empty:
        break

    if first_write:
        chunk_df.to_csv(output_file, index=False, mode="w")  # Write with headers
        first_write = False
    else:
        chunk_df.to_csv(output_file, index=False, mode="a", header=False)  # Append without headers

    rows_processed = len(chunk_df)
    del chunk_df

    processed_rows += rows_processed

    print(f"Processed {processed_rows} rows", end="\r")  # 'end="\r"' ensures it overwrites the previous line

    time.sleep(sleep_time)

print(f"Processing complete! Data saved to: {output_file}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Processing complete! Data saved to: processed_open_library_data_dump.csv


Try to process the 10000 rows of the data to check the conponents of the data

In [90]:
import pandas as pd
import json

output_file = "processed_open_library_data_dump.csv"
column_names = ["book_id", "revision", "timestamp", "json_data"]

sample_df = pd.read_csv(output_file, nrows=100000, names=column_names, header=None)


In [91]:
sample_df

,book_id,revision,timestamp,json_data
0,/books/OL10000299M,3,2011-04-27 14:29:38.147197,"{""publishers"": [""Stationery Office Books""], ""p..."
1,/books/OL10000402M,3,2011-04-27 11:28:52.494785,"{""publishers"": [""Stationery Office Books""], ""p..."
2,/books/OL10000661M,3,2011-04-25 22:07:29.549200,"{""publishers"": [""Stationery Office Books""], ""p..."
3,/books/OL10000852M,3,2011-04-27 04:54:31.855299,"{""publishers"": [""Stationery Office Books""], ""p..."
4,/books/OL10001003M,3,2011-06-08 04:28:03.025393,"{""publishers"": [""Stationery Office Books""], ""p..."
...,...,...,...,...
99995,/books/OL35738257M,2,2023-02-01 15:25:47.290397,"{""type"": {""key"": ""/type/edition""}, ""publish_da..."
99996,/books/OL357382M,7,2024-07-15 01:39:27.737325,"{""publishers"": [""St. Martin's Press""], ""identi..."
99997,/books/OL35738372M,6,2023-02-28 18:40:28.640066,"{""notes"": {""type"": ""/type/text"", ""value"": ""\""P..."
99998,/books/OL35738535M,1,2021-12-24 06:04:15.305092,"{""type"": {""key"": ""/type/edition""}, ""publish_da..."


In [92]:

sample_df["json_data"] = sample_df["json_data"].apply(lambda x: json.loads(x) if isinstance(x, str) else {})

normalized_df = pd.json_normalize(sample_df["json_data"])

final_df = pd.concat([sample_df.drop(columns=["json_data"]), normalized_df], axis=1)


In [93]:
 # read the processed data
final_df

,book_id,revision,timestamp,publishers,physical_format,title,number_of_pages,isbn_13,isbn_10,publish_date,...,identifiers.freebase,identifiers.bayerische_staatsbibliothek,identifiers.british_national_bibliography,identifiers.amazon.ca_asin,identifiers.zdb-id,identifiers.librivox,identifiers.bibliothèque_nationale_de_france,identifiers.project_gutenberg,identifiers.scribd,identifiers.isfdb
0,/books/OL10000299M,3,2011-04-27 14:29:38.147197,[Stationery Office Books],Paperback,Human Rights Bill [H.L.],22.0,[9780108360688],[0108360687],"December 3, 1997",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/books/OL10000402M,3,2011-04-27 11:28:52.494785,[Stationery Office Books],Paperback,Northern Ireland (Sentences) Bill,4.0,[9780108361821],[0108361829],"July 17, 1998",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/books/OL10000661M,3,2011-04-25 22:07:29.549200,[Stationery Office Books],Paperback,Bank of England Bill,8.0,[9780108365706],[0108365700],"March 6, 1998",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/books/OL10000852M,3,2011-04-27 04:54:31.855299,[Stationery Office Books],Paperback,Regional Development Agencies Bill,2.0,[9780108367625],[0108367622],"June 17, 1998",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/books/OL10001003M,3,2011-06-08 04:28:03.025393,[Stationery Office Books],Paperback,Northern Ireland Bill,12.0,[9780108369155],[0108369153],"October 21, 1998",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,/books/OL35738257M,2,2023-02-01 15:25:47.290397,[Chang jiang shao nian er tong chu ban she],NaN,Kuang shen,NaN,[9787556006380],[7556006387],2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99996,/books/OL357382M,7,2024-07-15 01:39:27.737325,[St. Martin's Press],NaN,World orders in the making,276.0,NaN,[0312215487],1998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99997,/books/OL35738372M,6,2023-02-28 18:40:28.640066,"[DK Publishing, DK Eyewitness Travel]",NaN,Top 10 Orlando,160.0,[9781465402776],[1465402772],2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99998,/books/OL35738535M,1,2021-12-24 06:04:15.305092,[Bei fang fu nü er tong chu ban she],NaN,Yong you jiu shi xing fu,NaN,[9787538588248],[7538588248],2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are still many dictionaries in the data, so we need to normalize the data again

convert the authors, works, isbn_13, isbn_10, publishers, and similar columns into plain text (string) format, the following changes were made:

In [94]:
if "isbn_13" in final_df.columns:
    final_df["isbn_13"] = final_df["isbn_13"].apply(lambda x: ", ".join(map(str, x)) if isinstance(x, list) else str(x) if pd.notna(x) else "")

if "isbn_10" in final_df.columns:
    final_df["isbn_10"] = final_df["isbn_10"].apply(lambda x: ", ".join(map(str, x)) if isinstance(x, list) else str(x) if pd.notna(x) else "")

if "publishers" in final_df.columns:
    final_df["publishers"] = final_df["publishers"].apply(lambda x: ", ".join(map(str, x)) if isinstance(x, list) else str(x) if pd.notna(x) else "")

if "oclc_numbers" in final_df.columns:
    final_df["oclc_numbers"] = final_df["oclc_numbers"].apply(lambda x: ", ".join(map(str, x)) if isinstance(x, list) else str(x) if pd.notna(x) else "")

# Extract 'key' values from lists of dictionaries
if "authors" in final_df.columns:
    final_df["authors"] = final_df["authors"].apply(lambda x: ", ".join(d["key"] for d in x if isinstance(d, dict) and "key" in d)
                                                    if isinstance(x, list) else str(x) if pd.notna(x) else "")

if "works" in final_df.columns:
    final_df["works"] = final_df["works"].apply(lambda x: ", ".join(d["key"] for d in x if isinstance(d, dict) and "key" in d)
                                                if isinstance(x, list) else str(x) if pd.notna(x) else "")

In [95]:
final_df

,book_id,revision,timestamp,publishers,physical_format,title,number_of_pages,isbn_13,isbn_10,publish_date,...,identifiers.freebase,identifiers.bayerische_staatsbibliothek,identifiers.british_national_bibliography,identifiers.amazon.ca_asin,identifiers.zdb-id,identifiers.librivox,identifiers.bibliothèque_nationale_de_france,identifiers.project_gutenberg,identifiers.scribd,identifiers.isfdb
0,/books/OL10000299M,3,2011-04-27 14:29:38.147197,Stationery Office Books,Paperback,Human Rights Bill [H.L.],22.0,9780108360688,0108360687,"December 3, 1997",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/books/OL10000402M,3,2011-04-27 11:28:52.494785,Stationery Office Books,Paperback,Northern Ireland (Sentences) Bill,4.0,9780108361821,0108361829,"July 17, 1998",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/books/OL10000661M,3,2011-04-25 22:07:29.549200,Stationery Office Books,Paperback,Bank of England Bill,8.0,9780108365706,0108365700,"March 6, 1998",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/books/OL10000852M,3,2011-04-27 04:54:31.855299,Stationery Office Books,Paperback,Regional Development Agencies Bill,2.0,9780108367625,0108367622,"June 17, 1998",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/books/OL10001003M,3,2011-06-08 04:28:03.025393,Stationery Office Books,Paperback,Northern Ireland Bill,12.0,9780108369155,0108369153,"October 21, 1998",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,/books/OL35738257M,2,2023-02-01 15:25:47.290397,Chang jiang shao nian er tong chu ban she,NaN,Kuang shen,NaN,9787556006380,7556006387,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99996,/books/OL357382M,7,2024-07-15 01:39:27.737325,St. Martin's Press,NaN,World orders in the making,276.0,,0312215487,1998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99997,/books/OL35738372M,6,2023-02-28 18:40:28.640066,"DK Publishing, DK Eyewitness Travel",NaN,Top 10 Orlando,160.0,9781465402776,1465402772,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99998,/books/OL35738535M,1,2021-12-24 06:04:15.305092,Bei fang fu nü er tong chu ban she,NaN,Yong you jiu shi xing fu,NaN,9787538588248,7538588248,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
# check the columns
final_df.columns

Index(['book_id', 'revision', 'timestamp', 'publishers', 'physical_format',
       'title', 'number_of_pages', 'isbn_13', 'isbn_10', 'publish_date',
       ...
       'identifiers.freebase', 'identifiers.bayerische_staatsbibliothek',
       'identifiers.british_national_bibliography',
       'identifiers.amazon.ca_asin', 'identifiers.zdb-id',
       'identifiers.librivox', 'identifiers.bibliothèque_nationale_de_france',
       'identifiers.project_gutenberg', 'identifiers.scribd',
       'identifiers.isfdb'],
      dtype='object', length=120)

In [97]:
final_df.dtypes

book_id                                         object
revision                                         int64
timestamp                                       object
publishers                                      object
physical_format                                 object
                                                 ...  
identifiers.librivox                            object
identifiers.bibliothèque_nationale_de_france    object
identifiers.project_gutenberg                   object
identifiers.scribd                              object
identifiers.isfdb                               object
Length: 120, dtype: object

In [98]:
main_columns = [
    "book_id", "title", "full_title", "isbn_13", "isbn_10", "key", "works",
    "publishers", "publish_date", "publish_country", "edition_name",
    "number_of_pages", "physical_format", "physical_dimensions", "weight",
    "authors", "by_statement",
    "subjects", "genres", "identifiers.goodreads",
    "identifiers.google", "identifiers.amazon", "identifiers.doi",
    "identifiers.wikidata", "identifiers.librarything", "identifiers.better_world_books",
    "url", "first_sentence", "description"
]

filtered_df = final_df[main_columns]

In [99]:
filtered_df

,book_id,title,full_title,isbn_13,isbn_10,key,works,publishers,publish_date,publish_country,...,identifiers.goodreads,identifiers.google,identifiers.amazon,identifiers.doi,identifiers.wikidata,identifiers.librarything,identifiers.better_world_books,url,first_sentence,description
0,/books/OL10000299M,Human Rights Bill [H.L.],NaN,9780108360688,0108360687,/books/OL10000299M,/works/OL14903285W,Stationery Office Books,"December 3, 1997",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/books/OL10000402M,Northern Ireland (Sentences) Bill,NaN,9780108361821,0108361829,/books/OL10000402M,/works/OL14902858W,Stationery Office Books,"July 17, 1998",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/books/OL10000661M,Bank of England Bill,NaN,9780108365706,0108365700,/books/OL10000661M,/works/OL14903105W,Stationery Office Books,"March 6, 1998",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/books/OL10000852M,Regional Development Agencies Bill,NaN,9780108367625,0108367622,/books/OL10000852M,/works/OL14903245W,Stationery Office Books,"June 17, 1998",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/books/OL10001003M,Northern Ireland Bill,NaN,9780108369155,0108369153,/books/OL10001003M,/works/OL14903191W,Stationery Office Books,"October 21, 1998",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,/books/OL35738257M,Kuang shen,Kuang shen Shen fen yi yun,9787556006380,7556006387,/books/OL35738257M,/works/OL19649579W,Chang jiang shao nian er tong chu ban she,2014,cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99996,/books/OL357382M,World orders in the making,NaN,,0312215487,/books/OL357382M,/works/OL18948578W,St. Martin's Press,1998,nyu,...,[1357099],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99997,/books/OL35738372M,Top 10 Orlando,Top 10 Orlando,9781465402776,1465402772,/books/OL35738372M,/works/OL21709257W,"DK Publishing, DK Eyewitness Travel",2014,nyu,...,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
99998,/books/OL35738535M,Yong you jiu shi xing fu,NaN,9787538588248,7538588248,/books/OL35738535M,/works/OL26451835W,Bei fang fu nü er tong chu ban she,2015,cc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Check the missing values in the data

In [114]:
print(filtered_df["book_id"].count())
print(filtered_df["identifiers.goodreads"].isna().sum())
print(filtered_df["isbn_13"].isna().sum())
print(filtered_df["isbn_10"].isna().sum())
print(filtered_df["works"].isna().sum())
print(filtered_df["publish_country"].isna().sum())
print(filtered_df["url"].isna().sum())

100000
86378
0
0
0
39930
98292


In [115]:
key_column=["book_id", "title", "full_title", "isbn_13", "isbn_10", "key", "works","publishers","publish_country","subjects", "genres", "identifiers.goodreads","identifiers.google", "identifiers.amazon", "first_sentence", "description"]
key_df=filtered_df[key_column].sample(100)
key_df

,book_id,title,full_title,isbn_13,isbn_10,key,works,publishers,publish_country,subjects,genres,identifiers.goodreads,identifiers.google,identifiers.amazon,first_sentence,description
27551,/books/OL16783561M,gramophone record library,NaN,,,/books/OL16783561M,/works/OL11924670W,Grafton,xxk,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67161,/books/OL261935M,Maternity care at public expense in six counti...,NaN,,,/books/OL261935M,/works/OL1588687W,U.S Govt. Print. Off.,dcu,"[Obstetrics, Mothers, Public health -- New Yor...",NaN,NaN,NaN,NaN,NaN,NaN
1983,/books/OL10458120M,"Bonnard, Peintre du Merveilleux",NaN,9780320058165,0320058166,/books/OL10458120M,/works/OL1136797W,French & European Pubns,NaN,"[Individual Artist, Art]",NaN,NaN,NaN,NaN,NaN,NaN
89545,/books/OL32799906M,Passage to Avalon,NaN,9780999511022,0999511025,/books/OL32799906M,/works/OL24710064W,Michael Thayer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33069,/books/OL1807333M,Raum und Volkstum,NaN,,340206930X,/books/OL1807333M,/works/OL4393102W,Aschendorff,gw,[Westphalia (Germany) -- Cultural policy.],NaN,[4629743],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65706,/books/OL25655137M,la mudez de las piedras,NaN,9788416234431,,/books/OL25655137M,/works/OL17084686W,Uno Editorial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60565,/books/OL24429509M,Zakon o svojinsko-pravnim odnosima,NaN,9788674420607,,/books/OL24429509M,/works/OL15463336W,JU Službeni list Crne Gore,mo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50999,/books/OL22264749M,Rally once again,NaN,,1852232226,/books/OL22264749M,/works/OL1809143W,"Crowood in association with Anthony Bird, The ...",xxk,"[United States -- History -- Civil War, 1861-1...",NaN,NaN,NaN,NaN,NaN,NaN
60163,/books/OL2433699M,The sin and the sinners,NaN,,,/books/OL2433699M,/works/OL3149476W,Jarrolds,enk,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Skimpy does not return anything

In [116]:
from skimpy import skim
key_df = pd.DataFrame(key_df).reset_index(drop=True)
skim(key_df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ Dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 100    │ │ string      │ 9     │                                                          │
│ │ Number of columns │ 13     │ │ object      │ 4     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                    All null                                                     │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column                                                              ┃ NA              ┃ NA %               ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩  │
│ │ identifiers.google                                                  │             100 │                100 │  │
│ │ identifiers.amazon                                                  │             100 │                100 │  │
│ │ first_sentence                                                      │             100 │                100 │  │
│ │ description                                                         │             100 │                100 │  │
│ └─────────────────────────────────────────────────────────────────────┴─────────────────┴────────────────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━┳━━━━┳━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┓  │
│ ┃           ┃    ┃      ┃           ┃           ┃           ┃           ┃ chars per ┃ words per ┃ total      ┃  │
│ ┃ column    ┃ NA ┃ NA % ┃ shortest  ┃ longest   ┃ min       ┃ max       ┃ row       ┃ row       ┃ words      ┃  │
│ ┡━━━━━━━━━━━╇━━━━╇━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━┩  │
│ │ book_id   │  0 │    0 │ /books/OL │ /books/OL │ /books/OL │ /books/OL │      17.8 │         1 │        100 │  │
│ │           │    │      │ 261935M   │ 16783561M │ 10330306M │ 35639242M │           │           │            │  │
│ │ title     │  0 │    0 │ Woman     │ War the   │ 3ds Max   │ la mudez  │      40.3 │         6 │        602 │  │
│ │           │    │      │           │ stumbling │ bang ding │ de las    │           │           │            │  │
│ │           │    │      │           │ -block of │ de yi shu │ piedras   │           │           │            │  │
│ │           │    │      │           │ a         │           │           │           │           │            │  │
│ │           │    │      │           │ Christian │           │           │           │           │            │  │
│ │           │    │      │           │ ; or, the │           │           │           │           │            │  │
│ │           │    │      │           │ absurdity │           │           │           │           │            │  │
│ │           │    │      │           │ of        │           │           │           │           │            │  │
│ │           │    │      │           │ defending │           │           │           │           │            │  │
│ │           │    │      │           │ religion  │           │           │           │           │            │  │
│ │           │    │      │           │ by the    │     

In [117]:
total_book_ids = filtered_df["book_id"].count()

unique_book_ids = filtered_df["book_id"].nunique()

# Print the results
print(f"Total book_id values: {total_book_ids}")
print(f"Unique book_id values: {unique_book_ids}")


Total book_id values: 100000
Unique book_id values: 100000


In [118]:
total_works = filtered_df["works"].count()

unique_works = filtered_df["works"].nunique()

print(f"Total works values: {total_works}")
print(f"Unique works values: {unique_works}")


Total works values: 100000
Unique works values: 92309


In [69]:
# skim the data
from skimpy import skim
filtered_df = pd.DataFrame(filtered_df)


# Sample 100 rows and reset index correctly
test_df = filtered_df.sample(100, random_state=42).reset_index(drop=True)
test_df

,book_id,title,full_title,isbn_13,isbn_10,key,works,publishers,publish_date,publish_country,...,identifiers.google,identifiers.amazon,identifiers.doi,identifiers.wikidata,identifiers.librarything,identifiers.better_world_books,url,first_sentence,description,table_of_contents
0,/books/OL11458883M,Hotel & Restaurant Dictionary Fachworterbuch ...,NaN,9780828814782,0828814783,/books/OL11458883M,/works/OL9250608W,French & European Pubns,"October 1, 1981",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/books/OL11088269M,Mary Higgins Clark Presents Malice Domestic 2,NaN,9780787105693,0787105694,/books/OL11088269M,/works/OL31189742W,Audio Literature,July 1996,NaN,...,NaN,NaN,NaN,NaN,[359866],NaN,NaN,NaN,NaN,NaN
2,/books/OL10401622M,The 2007-2012 Outlook for Indian Breads in Japan,NaN,9780497467661,0497467666,/books/OL10401622M,/works/OL3309893W,"ICON Group International, Inc.","September 28, 2006",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/books/OL11102775M,Pregnancy-Care & Physiology,NaN,9780788301896,0788301896,/books/OL11102775M,/works/OL9542721W,Abbe Pub Assn of Washington Dc,June 1994,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/books/OL11050043M,Harvard Planner,NaN,9780865020580,0865020582,/books/OL11050043M,/works/OL9155050W,Executive.org,"December 21, 1994",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,/books/OL10887803M,2000 Import and Export Market for Nickel in Vi...,NaN,9780597709159,0597709157,/books/OL10887803M,/works/OL9461531W,Icon Group International,January 2001,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,/books/OL12136146M,Win HMO/PPO Combo Pack with CDROM,NaN,9781563633201,1563633205,/books/OL12136146M,/works/OL8539593W,Thomson Healthcare,October 1998,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,/books/OL11824378M,The Rover Boys at School,NaN,9781421841328,1421841320,/books/OL11824378M,/works/OL234952W,1st World Library - Literary Society,"June 15, 2007",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,/books/OL11754548M,The Inside Of The Cup,NaN,9781414284859,1414284853,/books/OL11754548M,/works/OL18521604W,IndyPublish.com,"May 31, 2004",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
skim(filtered_df.sample(100).reset_index(drop=True))

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ Dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 100    │ │ string      │ 15    │                                                          │
│ │ Number of columns │ 26     │ │ object      │ 10    │                                                          │
│ └───────────────────┴────────┘ │ float64     │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                    All null                                                     │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column                                                                ┃ NA             ┃ NA %              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩  │
│ │ full_title                                                            │            100 │               100 │  │
│ │ genres                                                                │            100 │               100 │  │
│ │ identifiers.google                                                    │            100 │               100 │  │
│ │ identifiers.amazon                                                    │            100 │               100 │  │
│ │ identifiers.wikidata                                                  │            100 │               100 │  │
│ │ identifiers.better_world_books                                        │            100 │               100 │  │
│ │ url                                                                   │            100 │               100 │  │
│ │ first_sentence                                                        │            100 │               100 │  │
│ │ description                                                           │            100 │               100 │  │
│ │ table_of_contents                                                     │            100 │               100 │  │
│ └───────────────────────────────────────────────────────────────────────┴────────────────┴───────────────────┘  │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column                ┃ NA   ┃ NA %   ┃ mean     ┃ sd      ┃ p0   ┃ p25   ┃ p50   ┃ p75  ┃ p100  ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━━━┩  │
│ │ number_of_pages       │   36 │     36 │    169.8 │   141.3 │    4 │    60 │   149 │  208 │   672 │ ▇█▂▁ ▁  │  │
│ └───────────────────────┴──────┴────────┴──────────┴─────────┴──────┴───────┴───────┴──────┴───────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━┳━━━━┳━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┓  │
│ ┃           ┃    ┃      ┃           ┃           ┃           ┃           ┃ chars per ┃ words per ┃ total      ┃  │
│ ┃ column    ┃ NA ┃ NA % ┃ shortest  ┃ longest   ┃ min       ┃ max       ┃ row       ┃ row       ┃ words      ┃  │
│ ┡━━━━━━━━━━━╇━━━━╇━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━

In [71]:
print(filtered_df.columns.duplicated().any())  # Check for duplicate column names


False


In [62]:
sampled_df = filtered_df.sample(100, random_state=42)

In [60]:
filtered_df = filtered_df.reset_index(drop=True)
skimpy.skim(filtered_df.sample(10))  # Use only 100 rows

KeyError: np.int64(0)

In [ ]:

# Run skimpy.skim() safely
skimpy.skim(final_df)

In [12]:
import skimpy
skimpy.skim(final_df)

AttributeError: 'DataFrame' object has no attribute 'dtype'